# Søk i tidsskrift

In [1]:
from multiprocess import Pool

p = Pool(10)

In [2]:
import json
import sqlite3
import requests
import pandas as pd
import dask
import pandas as pd
import dhlab.module_update as mu
mu.update('mods_meta')
import mods_meta as mm
import dhlab.nbtext as nb
from dhlab.nbtokenizer import tokenize
import re
from IPython.display import Markdown
from collections import Counter
import parallellkonk as pk

Updated file `C:\Users\larsj\Documents\GitHub\bokhylla_api_medietyper\mods_meta.py`

### fjern ellipser i pandarammer

In [3]:
pd.set_option('display.max_colwidth', None)

### Hjelpefunksjoner

In [4]:
def nb_search(
    term = '', 
    creator = None, 
    number = 50, 
    page = 0, 
    title = None,
    mediatype = None, 
    lang = None,
    period = None,
    random = False
):
    """Søk etter term og få ut json"""
    
    # max number of hits pr. page
    # page has also a max of 50 - not checked
    
    number = min(number, 50)
    
    filters = []
    aq = []
    

    params = {
        'page':page, 
        'size':number,
        'random': random
    }
    
    if lang != None:
        aq.append('languages:{lang}'.format(lang = lang ))
    
    if title != None:
        filters.append('title:{title}'.format(title = title))
        
    if creator != None:
        filters.append('creator:{c}'.format(c=creator))
    
    if mediatype != None:
        filters.append('mediatype:{mediatype}'.format(mediatype=mediatype))
    
    if period != None:
        filters.append('date:[{date_from} TO {date_to}]'.format(date_from = period[0], date_to = period[1]))
    
    if filters != []:
        params['filter'] = filters
    
    if aq != []:
        params['aq'] = aq
        
    if term != None:
        params['q'] = term
    
    r = requests.get("https://api.nb.no:443/catalog/v1/items", params = params)
    return r.json()

def find_urns_sesam(term = None, creator = None, number=50, page=0, lang = None, title = None, mediatype = None, 
                    period = None, random = False):
    """generates urns/sesamid from item search"""
    x = nb_search(
        term = term, creator = creator, title = title,
        number = number, page = page, mediatype=mediatype, period = period, 
        lang = lang,
        random = random
    )
    try:
        sesamid =[f['id'] for f in x['_embedded']['items']]
    except:
        sesamid = []
    return sesamid


def find_items(term = None, 
               creator = None, 
               number = 100, 
               lang = None, 
               title = None, 
               mediatype = None, 
                period = None
              ):
    """find ids from a spec"""
    pages = number // 50
    lastnumbers = number % 50
    print(pages, lastnumbers)
    res = []
    # fetch ids from pages, max hits
    for page in range(pages):
        x = nb_search(
            term = term, creator = creator, title = title,
            number = 50, page = page, mediatype = mediatype, period = period, 
            lang = lang
        )
        res += [f['id'] for f in x['_embedded']['items']]

    # fetch ids from last page
    x = nb_search(
            term = term, creator = creator, title = title,
            number = lastnumbers, page = pages + 1, mediatype=mediatype, period = period, 
            lang = lang)
    res += [f['id'] for f in x['_embedded']['items']]
                  
    return res

def fetch_keys(m, path, delimiter = "/", res = [], start_list = '#'):
    """path /-delimited string, return res if fails, array indices indicaed with start_list"""
    
    # get the sequence of path elements
    path = path.split(delimiter)
    
    x = m
    try:
        for i in range(0, len(path)):
            if path[i].startswith(start_list):
                # then the item is an array selector
                index = int(path[i].split(start_list)[-1])
                x = x[index]
            else:
                x = x[path[i]]
    except KeyError:
        x = res
    return x
        
def find_item(data, item):
    res = []
    if isinstance(data, list):
        print('list', data)
        for subdata in data:
            res += find_item(subdata, item)
    elif isinstance(data, dict):
        for key in data:
            if item in data[key]:
                print('dictvalue',key, data[key])
                res.append(data[key][item])
            else:
                res += find_item(data[key], item)
    return res

def metadata(id):
    r = requests.get("https://api.nb.no:443/catalog/v1/items/" + str(id))

    d = r.json()
    res = {
        'title': fetch_keys(d, 'metadata/title'),
        'contr':[(fetch_keys(x, 'name'), fetch_keys(x, 'roles/#0/name')) for x in fetch_keys(d, 'metadata/people')],
        'pages': fetch_keys(d, 'metadata/physicalDescription/extent'),
        'urn': fetch_keys(d, 'metadata/identifiers/urn'),
        'year': fetch_keys(d, 'metadata/originInfo/issued'),
        'topics':fetch_keys(d, 'metadata/subject/topics'),
        'genres': fetch_keys(d, 'metadata/genres'),
        'target_group':fetch_keys(d, 'metadata/targetAudienceNotes')
                            
    }

    return res

def get_konks(urn, phrase, window = 500, n = 1000):
    import requests
    querystring = '"'+ phrase +'"' 
    query = {
        'q':querystring,
        'fragments': n,
        'fragSize':window
       
    }
    r = requests.get("https://api.nb.no/catalog/v1/items/{urn}/contentfragments".format(urn=urn), params = query)
    res = r.json()
    results = []
    try:
        for x in res['contentFragments']:
            pid = x['pageid']
            hit = x['text']
            splits = hit.split('<em>')
            s2 = splits[1].split('</em>')
            before = splits[0]
            word = s2[0]
            after = s2[1]
            results.append({'urn': urn, 'before': before, 'word':word, 'after':after})
    except:
        True
    return results

def get_konkordanser(word = '', urns = None, window = 500, n = 1000):
    konks = []
    for u in urns:
        konks += get_konks(u, word, window = window, n = n)
    return konks

In [5]:
def count_from_conc(concordance):
    """From a concordance, count the words in it. Assume konks reside a dataframe with columns 'after' and 'before'"""
    word = concordance['word'][0]
    return nb.frame_sort(nb.frame(Counter(tokenize(' '.join(concordance['after'].values + concordance['before'].values))), word))

def count_from_conc_window(concordance, before = 5, after = 5):
    """From a concordance, count the words in it. Assume konks reside a dataframe with columns 'after' and 'before'"""
    word = concordance['word'][0]
    # join before and after
    return nb.frame_sort(
            nb.frame(
                Counter(
                    [tokens for toklists in [tokenize(c)[:after] for c in concordance['after'].values] for tokens in toklists] 
                    + 
                    [tokens for toklists in [tokenize(c)[-before:] for c in concordance['before'].values] for tokens in toklists]
                )
            ), 
            word
        )

# Søk i tidsskrift

In [6]:
rand = find_urns_sesam('',  number = 50, random = True, mediatype = 'tidsskrift', title = 'blikk')

In [7]:
konk = pd.DataFrame(get_konkordanser('vi', urns = rand, window = 500))

In [8]:
konk

,urn,before,word,after
0,274c194c0670b84625233d15dbc2a204,ungdom er ei ekstra sårbar gruppe: «[...] de lurer på mange hjerteskjærende ting. For eksempel fikk,vi,spørsmålet ‘hvordan kan jeg stoppe å være homofil?’ Mange skulle ønske at de var noe annet enn
1,274c194c0670b84625233d15dbc2a204,som tenker helhetlig.,Vi,trenger trojanske hester som løfter lhbt-temaet inn på arenaer
2,274c194c0670b84625233d15dbc2a204,"Skeiv verden og Skeiv ungdom skal utradere skammen og mobbingen alene, er uansvarlig av regjeringen.",Vi,forventer at de rødgrønne nå lager en konkret og helhetlig lhbt-plan der tiltakene i skolen blir
3,274c194c0670b84625233d15dbc2a204,år siden homoforbudet i straffelo- flere radiohits bak seg. ven ble opphevet. Med oss på laget har,vi,"fått Mer sang blir det når Norges lekreste homseden største homoforkjemperen i Norge, Kim Fri- kor,"
4,274c194c0670b84625233d15dbc2a204,"Morten Skogmus og Klatremus er som kjent venner. - Forholdet deres er ikke helt definert, men åpent.",Vi,"får ikke vite om deres sivile status, og det kan man bruke til sin egen fordel. Klatremus"
...,...,...,...,...
6228,15978476302695a4b857b76456d7945b,"sier han. gg "" Hamlet, trommer B .< Morsomt seg til en hybrid av funk, jazz-ele- Så far",vi,"håpe Sugarpops-budskapet menter og jungle, alt innpakket i nådde fram til homsene. I hvertfall 90-tallslyd,"
6229,15978476302695a4b857b76456d7945b,Sugarpops at det var mortydelig satte pris på. - Det var et somt å spille på Castro. -,Vi,"kjengodt publikum, sier Malika, grup- ner mange i miljøet så det er vår .yl|| .|B pas eneste"
6230,15978476302695a4b857b76456d7945b,en i dykker- re fra Amsterdam og Berlin ker til at våre deiligste og Store under- drakt og,vi,så også en kjekk - helt til du spaserer ut i skitneste fantasier kan leves holdninasiiODO- sikkerhetsvakt
6231,15978476302695a4b857b76456d7945b,nnom uten synlig kles- Oslo som for tida definitivt og pusset opp huset med bar utsida møtte,vi,"en fyr som ode, er det alltid mye gir en følelse av å være iet nede, skikkelig pisserenne,"


In [9]:
coll = count_from_conc_window(konk, before = 10, after = 0)

In [10]:
coll.sum()

0    57164
dtype: int64